<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.62
    Uninstalling yfinance-0.2.62:
      Successfully uninstalled yfinance-0.2.62
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-12 09:34:04
-------------------
qualified stocks: 90
with latest results: 29
still star stocks: 17
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  4.16 L
Current:  1.36 C
-------------------
Today PnL: 2.11 K (0.02%)
Current PnL: -13.78 L (-9.6%)
CY Booked + Current PnL: -7.70 L (-5.36%)
-------------------
Total profit:  3.92 L
Total loss:  -17.70 L
-------------------
Total Booked + Current PnL: 19.28 L (16.55%)
Total Booked PnL: 33.06 L (28.37%)
Curr Year Booked PnL: 6.08 L (4.47%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.05 C
Est FTT PnL: 69.29 L (50.97%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.88%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LICI,785.05,983.0,-2.96,H-LC,86.67,237758.0,43066.0,6015.0,-0.63,22.12,2.53,25.21,34.0,7.16,1.80,31.47,XY25,NTT,INSURANCE
59,RELIANCE,1291.83,1526.0,6.50,X-LC,26.67,158442.0,17633.0,7890.0,0.32,12.52,4.98,18.13,19.0,2.23,1.20,25.11,XY25,NTT,REFINERIES
3,ADANIPORTS,1103.69,1583.0,3.60,M-LC,54.44,104008.0,25646.0,8383.0,0.58,32.73,8.06,43.43,77.0,3.06,0.79,39.11,XY24,NTT,MISC
81,UJJIVANSFB,52.77,53.0,54.03,M-SC,83.33,133758.0,-8721.0,9336.0,1.08,-6.12,6.98,0.44,241.0,-0.93,1.02,57.87,OX40N,NTT,BANKS
75,TATAELXSI,7332.28,7332.0,-12.62,X-MC,56.67,91770.0,-10882.0,10875.0,-1.05,-10.60,11.85,-0.00,55.0,-1.00,0.70,39.50,OX40N,NTT,IT


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,NATIONALUM,189.63,247.44,-36.43,L-SC,72.22,100509.0,5.0,30635.0,-0.19,0.01,30.48,30.49,246.0,0.00,0.76,32.41,X5K,ATH,METALS
13,BERGEPAINT,561.33,680.00,-16.65,H-MC,44.44,228400.0,1061.0,47005.0,-0.57,0.47,20.58,21.14,118.0,0.02,1.73,27.88,XY24,NTT,PAINTS
33,HINDALCO,651.95,761.55,-12.54,M-LC,17.78,104384.0,72.0,17463.0,-0.43,0.07,16.73,16.81,71.0,0.00,0.79,16.13,X5K,ATH,METALS
30,HAVELLS,1588.02,2069.16,-4.62,X-LC,20.00,246449.0,-1282.0,76350.0,0.04,-0.52,30.98,30.30,50.0,-0.02,1.87,12.71,X40,ATH,ELECTRICAL


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TANLA,917.30,1963.11,-26.16,H-SC,98.89,181408.0,-58007.0,330961.0,11.63,-24.23,182.44,114.01,127.0,-0.18,1.38,66.38,AR,ATH,IT
84,VALIANTORG,512.64,838.00,-405.68,H-SC,100.00,130000.0,-3286.0,87880.0,3.38,-2.47,67.60,63.47,144.0,-0.04,0.99,117.91,XR,NTT,CHEMICALS
66,SFL,1016.35,1287.00,288.73,H-SC,40.00,175427.0,-86791.0,156621.0,2.16,-33.10,89.28,26.63,146.0,-0.55,1.33,9.07,XY24,NTT,MISC
6,ASIANPAINT,2961.56,4250.00,-23.00,X-LC,7.78,153204.0,-48182.0,135800.0,2.00,-23.93,88.64,43.51,5.0,-0.35,1.16,6.65,X40,BTT,PAINTS
88,WHIRLPOOL,1167.49,2270.00,-30.75,M-SC,55.56,201743.0,30122.0,131940.0,1.70,17.55,65.40,94.43,191.0,0.23,1.53,48.90,XR,NTT,DURABLES


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,711.66,986.0,NaN,NaN,1.11,67846.0,-88008.0,148087.0,-1.96,-56.47,218.27,38.55,190.0,-0.59,0.51,3.89,XY24,NTT,MISC
46,JPPOWER,18.73,26.2,-33.51,L-SC,95.56,135330.0,-6512.0,63077.0,-1.92,-4.59,46.61,39.88,256.0,-0.10,1.03,37.15,XY24,NTT,POWER
21,COFFEEDAY,59.14,80.0,-51.71,L-SC,93.33,70426.0,-43123.0,83173.0,-1.74,-37.98,118.10,35.27,260.0,-0.52,0.53,72.37,XR,NTT,HOTELS
49,LAOPALA,369.40,464.0,128.40,H-SC,66.67,72509.0,-28337.0,54164.0,-1.64,-28.10,74.70,25.61,133.0,-0.52,0.55,30.77,AR,NTT,CERAMICS
42,INFY,1461.46,2275.0,-10.27,X-LC,18.89,287546.0,25945.0,119677.0,-1.51,9.92,41.62,55.67,2.0,0.22,2.18,16.63,X40,BTT,IT


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,UJJIVANSFB,52.77,53.0,54.03,M-SC,83.33,133758.0,-8721.0,9336.0,1.08,-6.12,6.98,0.44,241.0,-0.93,1.02,57.87,OX40N,NTT,BANKS
73,SYMPHONY,1306.42,1306.0,-27.09,M-SC,3.33,155235.0,-15906.0,15849.0,-0.40,-9.29,10.21,-0.03,189.0,-1.00,1.18,11.57,OX40N,NTT,DURABLES
75,TATAELXSI,7332.28,7332.0,-12.62,X-MC,56.67,91770.0,-10882.0,10875.0,-1.05,-10.60,11.85,-0.00,55.0,-1.00,0.70,39.50,OX40N,NTT,IT
86,VIPIND,488.80,489.0,-954.99,H-SC,92.22,81432.0,-13884.0,13925.0,0.97,-14.57,17.10,0.04,151.0,-1.00,0.62,60.62,OX40N,NTT,MISC
47,KANSAINER,299.63,340.0,-67.67,H-SC,6.67,225675.0,-43992.0,80318.0,-0.02,-16.31,35.59,13.47,143.0,-0.55,1.71,12.96,XY24,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TCS,3794.03,4998.00,-17.12,X-LC,8.89,270387.0,-25547.0,119457.0,-0.16,-8.63,44.18,31.73,1.0,-0.21,2.05,8.23,X40,BTT,IT
42,INFY,1461.46,2275.00,-10.27,X-LC,18.89,287546.0,25945.0,119677.0,-1.51,9.92,41.62,55.67,2.0,0.22,2.18,16.63,X40,BTT,IT
51,LTIM,5564.16,7197.33,0.70,X-LC,62.22,193788.0,-6522.0,65307.0,-0.20,-3.26,33.70,29.35,3.0,-0.10,1.47,33.88,X200,ATH,IT
6,ASIANPAINT,2961.56,4250.00,-23.00,X-LC,7.78,153204.0,-48182.0,135800.0,2.00,-23.93,88.64,43.51,5.0,-0.35,1.16,6.65,X40,BTT,PAINTS
34,HINDUNILVR,2413.81,2723.00,-15.33,X-LC,15.56,265787.0,-4560.0,39177.0,-0.14,-1.69,14.74,12.81,6.0,-0.12,2.02,10.00,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,UNIONBANK,123.87,163.0,-0.31,M-LC,87.78,173438.0,32598.0,11898.0,-0.18,23.15,6.86,31.59,88.0,2.74,1.32,50.66,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,3.60,M-LC,54.44,104008.0,25646.0,8383.0,0.58,32.73,8.06,43.43,77.0,3.06,0.79,39.11,XY24,NTT,MISC
50,LICI,785.05,983.0,-2.96,H-LC,86.67,237758.0,43066.0,6015.0,-0.63,22.12,2.53,25.21,34.0,7.16,1.80,31.47,XY25,NTT,INSURANCE


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,130.64,228.00,100.99,M-MC,71.11,228199.0,3237.0,164417.0,-0.47,1.44,72.05,74.53,183.0,0.02,1.73,32.49,XY24,BTT,STEEL
35,HINDZINC,514.80,744.74,40.53,M-LC,91.11,105100.0,2140.0,43848.0,0.95,2.08,41.72,44.67,68.0,0.05,0.80,35.63,X5K,ATH,METALS
17,CAMS,3643.00,5250.99,4.67,H-SC,79.44,117597.0,15593.0,29435.0,-0.68,15.29,25.03,44.14,125.0,0.53,0.89,34.01,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,3.60,M-LC,54.44,104008.0,25646.0,8383.0,0.58,32.73,8.06,43.43,77.0,3.06,0.79,39.11,XY24,NTT,MISC
32,HEROMOTOCO,4311.81,6039.03,0.95,H-MC,63.33,154000.0,3087.0,57365.0,-0.23,2.05,37.25,40.06,101.0,0.05,1.17,25.96,AR,ATH,AUTO


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,WHIRLPOOL,1167.49,2270.00,-30.75,M-SC,55.56,201743.0,30122.0,131940.0,1.70,17.55,65.40,94.43,191.0,0.23,1.53,48.90,XR,NTT,DURABLES
11,BANKINDIA,116.91,190.00,-28.22,M-MC,75.56,191558.0,11750.0,100664.0,0.48,6.53,52.55,62.52,168.0,0.12,1.45,37.85,XR,NTT,BANKS
89,WIPRO,243.46,420.00,-6.99,M-LC,16.67,159433.0,8488.0,100969.0,-0.68,5.62,63.33,72.51,70.0,0.08,1.21,9.99,XR,NTT,IT
39,INDIAMART,2327.09,4871.06,-49.41,H-SC,58.89,131954.0,8618.0,126214.0,-0.34,6.99,95.65,109.32,122.0,0.07,1.00,30.36,AR,ATH,MISC
32,HEROMOTOCO,4311.81,6039.03,0.95,H-MC,63.33,154000.0,3087.0,57365.0,-0.23,2.05,37.25,40.06,101.0,0.05,1.17,25.96,AR,ATH,AUTO


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,59.14,80.00,-51.71,L-SC,93.33,70426.0,-43123.0,83173.0,-1.74,-37.98,118.10,35.27,260.0,-0.52,0.53,72.37,XR,NTT,HOTELS
52,MASFIN,326.60,399.50,-20.96,H-SC,70.00,90060.0,-7920.0,29792.0,-0.40,-8.08,33.08,22.32,138.0,-0.27,0.68,30.62,XR,ATH,FINANCE
3,ADANIPORTS,1103.69,1583.00,3.60,M-LC,54.44,104008.0,25646.0,8383.0,0.58,32.73,8.06,43.43,77.0,3.06,0.79,39.11,XY24,NTT,MISC
35,HINDZINC,514.80,744.74,40.53,M-LC,91.11,105100.0,2140.0,43848.0,0.95,2.08,41.72,44.67,68.0,0.05,0.80,35.63,X5K,ATH,METALS
72,SURYODAY,189.78,240.00,64.47,H-SC,90.00,117196.0,-32730.0,72404.0,-1.39,-21.83,61.78,26.46,141.0,-0.45,0.89,50.14,XR,NTT,BANKS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VALIANTORG,512.64,838.00,-405.68,H-SC,100.00,130000.0,-3286.0,87880.0,3.38,-2.47,67.60,63.47,144.0,-0.04,0.99,117.91,XR,NTT,CHEMICALS
74,TANLA,917.30,1963.11,-26.16,H-SC,98.89,181408.0,-58007.0,330961.0,11.63,-24.23,182.44,114.01,127.0,-0.18,1.38,66.38,AR,ATH,IT
7,ASIANTILES,75.41,137.00,6883.33,M-SC,97.78,76630.0,-17180.0,93795.0,-0.85,-18.31,122.40,81.67,235.0,-0.18,0.58,48.40,XR,NTT,CERAMICS
78,TITAGARH,1097.23,1548.00,-6.68,H-SC,96.67,163861.0,-28154.0,107034.0,-0.81,-14.66,65.32,41.08,152.0,-0.26,1.24,38.69,XY24,NTT,ENGINEERING
46,JPPOWER,18.73,26.20,-33.51,L-SC,95.56,135330.0,-6512.0,63077.0,-1.92,-4.59,46.61,39.88,256.0,-0.10,1.03,37.15,XY24,NTT,POWER


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.81
1,25,43.63
2,50,72.66


In [21]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     32.23
XY25     15.43
XR       11.08
X40      11.01
OX40N     8.57
X40N      8.51
AR        8.17
X5K       2.35
X200      1.47
SR        1.17
Name: CurrAlloc%, dtype: float64

In [22]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.20
X-LC    18.22
M-SC    14.42
M-LC    10.16
H-MC     9.03
H-LC     7.93
X-MC     6.21
M-MC     5.86
L-SC     3.26
X-SC     2.82
L-LC     1.13
Name: CurrAlloc%, dtype: float64

In [23]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.21,-1.09,34.73
IT,11.35,-11.01,67.37
BANKS,8.64,-6.95,50.05
MISC,6.65,-23.28,71.49
FINANCE,6.45,-18.23,63.72
PAINTS,5.66,-16.88,39.92
INSURANCE,5.25,3.68,25.06
ELECTRICAL,5.19,-2.75,47.32
HEALTHCARE,4.76,-2.78,32.80


In [24]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2682411.0,25
XR,1098955.0,13
AR,992986.0,9
X40,596483.0,8
XY25,497185.0,10
X40N,409240.0,8
OX40N,304159.0,11
SR,190776.0,2
X5K,91946.0,3


In [25]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1836065.0,18
M-SC,1426777.0,18
X-LC,810144.0,11
M-MC,571407.0,5
H-MC,537403.0,7
M-LC,376161.0,8
X-MC,327712.0,5
L-SC,298046.0,5
X-SC,262328.0,4


In [26]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,881927.0,6
M-SC,XY24,567531.0,5
H-SC,AR,511339.0,3
X-LC,X40,451284.0,4
M-MC,XY24,385981.0,3
M-SC,XR,339390.0,4
H-MC,XY24,221050.0,2
M-LC,XY24,201058.0,4
H-SC,XR,190076.0,3


In [27]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
